In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from datasets import load_dataset
import random
from nnsight import LanguageModel
import torch as t
from torch import nn
from attribution import patching_effect
from dictionary_learning import AutoEncoder, ActivationBuffer
from dictionary_learning.dictionary import IdentityDict
# from dictionary_learning.interp import examine_dimension
# no circuitsviz; no z standard library
# from dictionary_learning.utils import hf_dataset_to_generator
from tqdm import tqdm
import gc

DEBUGGING = False

if DEBUGGING:
    tracer_kwargs = dict(scan=True, validate=True)
else:
    tracer_kwargs = dict(scan=False, validate=False)

# model hyperparameters
# DEVICE = 'cuda:0'
DEVICE = "cpu"
model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map=DEVICE, dispatch=True)
activation_dim = 512

/common/home/users/x/xinrose.lin.2020/.conda/envs/fcs-exp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset hyperparameters
dataset = load_dataset("LabHC/bias_in_bios")
profession_dict = {'professor' : 21, 'nurse' : 13}
male_prof = 'professor'
female_prof = 'nurse'

# data preparation hyperparameters
# batch_size = 1024
batch_size = 8
SEED = 42

# To fit on 24GB VRAM GPU, I set the next 2 default batch_sizes to 64
def get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        n = min([len(neg), len(pos)])
        neg, pos = neg[:n], pos[:n]
        data = neg + pos
        labels = [0]*n + [1]*n
        idxs = list(range(2*n))
        random.Random(seed).shuffle(idxs)
        data, labels = [data[i] for i in idxs], [labels[i] for i in idxs]
        true_labels = spurious_labels = labels
    else:
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        n = min([len(neg_neg), len(neg_pos), len(pos_neg), len(pos_pos)])
        neg_neg, neg_pos, pos_neg, pos_pos = neg_neg[:n], neg_pos[:n], pos_neg[:n], pos_pos[:n]
        data = neg_neg + neg_pos + pos_neg + pos_pos
        true_labels     = [0]*n + [0]*n + [1]*n + [1]*n
        spurious_labels = [0]*n + [1]*n + [0]*n + [1]*n
        idxs = list(range(4*n))
        random.Random(seed).shuffle(idxs)
        data, true_labels, spurious_labels = [data[i] for i in idxs], [true_labels[i] for i in idxs], [spurious_labels[i] for i in idxs]

    batches = [
        (data[i:i+batch_size], t.tensor(true_labels[i:i+batch_size], device=DEVICE), t.tensor(spurious_labels[i:i+batch_size], device=DEVICE)) for i in range(0, len(data), batch_size)
    ]

    return batches

def get_subgroups(train=True, ambiguous=True, batch_size=128, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_labels, pos_labels = (0, 0), (1, 1)
        subgroups = [(neg, neg_labels), (pos, pos_labels)]
    else:
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_neg_labels, neg_pos_labels, pos_neg_labels, pos_pos_labels = (0, 0), (0, 1), (1, 0), (1, 1)
        subgroups = [(neg_neg, neg_neg_labels), (neg_pos, neg_pos_labels), (pos_neg, pos_neg_labels), (pos_pos, pos_pos_labels)]
    
    out = {}
    for data, label_profile in subgroups:
        out[label_profile] = []
        for i in range(0, len(data), batch_size):
            text = data[i:i+batch_size]
            out[label_profile].append(
                (
                    text,
                    t.tensor([label_profile[0]]*len(text), device=DEVICE),
                    t.tensor([label_profile[1]]*len(text), device=DEVICE)
                )
            )
    return out

In [3]:
# probe training hyperparameters
from torch import nn
layer = 4 # model layer for attaching linear classification head

class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits

def train_probe(get_acts, label_idx=0, batches=get_data(), lr=1e-2, epochs=1, dim=512, seed=SEED):
    t.manual_seed(seed)
    probe = Probe(dim).to(DEVICE)
    print('probe')
    optimizer = t.optim.AdamW(probe.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    losses = []
    for epoch in range(epochs):
        for batch in tqdm(batches):
            
            text = batch[0]
            # print(text)
            labels = batch[label_idx+1] 
            acts = get_acts(text)
            # print(acts.shape) (8, 512) (activations for a batch of 8 datapoints)
            logits = probe(acts)
            loss = criterion(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    return probe, losses

def test_probe(probe, get_acts, label_idx=0, batches=get_data(train=False), seed=SEED):
    with t.no_grad():
        corrects = []

        for batch in batches:
            text = batch[0]
            labels = batch[label_idx+1]
            acts = get_acts(text)
            logits = probe(acts)
            preds = (logits > 0.0).long()
            corrects.append((preds == labels).float())
        return t.cat(corrects).mean().item()
    
def get_acts(text):
    with t.no_grad(): 
        with model.trace(text, **tracer_kwargs):
            # print(text)
            attn_mask = model.inputs[1]['attention_mask']
            acts = model.gpt_neox.layers[layer].output[0]
            acts = acts * attn_mask[:, :, None]
            acts = acts.sum(1) / attn_mask.sum(1)[:, None]
            acts = acts.save()
        return acts.value

KeyboardInterrupt: 

In [ ]:
def get_acts(text):
    with t.no_grad(): 
        with model.trace(text, **tracer_kwargs):
            # print(text)
            attn_mask = model.inputs[1]['attention_mask']
            acts = model.gpt_neox.layers[layer].output[0]
            acts = acts * attn_mask[:, :, None]
            acts = acts.sum(1) / attn_mask.sum(1)[:, None]
            acts = acts.save()
        return acts.value
    

In [ ]:
batches = get_data(ambiguous=False)
# (text, profession, gender)

In [6]:
print('batches: ', len(batches))
print('each batch: ', len(batches[0][0]), 'data points in tuples of (text, gender, )')
print(batches[0][0][3])
print(f'batch 1 sample: {batches[0][0]} \n {batches[0][1]} \n {batches[0][2]}')
print(f'batch dtype: {len(batches[0])}, {type(batches[0])}, {type(batches[0][1])} \n str : {type(batches[0][0])}')

batches:  564
each batch:  8 data points in tuples of (text, gender, )
After graduating from the University of San Francisco she went on to explore different areas of nursing including, orthopedic trauma/surgery, emergency room, critical care and pediatric intensive care unit. She is also a Public Health Nurse with an expertise in working with multicultural patients as they intersect within a healthcare setting.
batch 1 sample: ['She graduated with honors in 2006. Having more than 11 years of diverse experiences, especially in NURSE PRACTITIONER, Sharon R Hoskin affiliates with many hospitals including Ut Southwestern University Hospital St Paul, Ut Southwestern University Hospital-zale Lipshy, and cooperates with other doctors and specialists in medical group The University Of Texas Southwestern Medical Center At Dallas. Call Sharon R Hoskin on phone number (214) 648-1454 for more information and advises or to book an appointment.', 'Chirag Patel’s long-term research goal is to addres

In [ ]:
# oracle, _ = train_probe(get_acts, label_idx=0, batches=get_data(ambiguous=False))
# print("ambiguous test accuracy", test_probe(oracle, get_acts, label_idx=0))

In [7]:
## train oracle, with balanced set
oracle, _ = train_probe(get_acts, label_idx=0, batches=get_data(ambiguous=False))
print("ambiguous test accuracy", test_probe(oracle, get_acts, label_idx=0))
## test with balanced set
batches = get_data(train=False, ambiguous=False)
## test with labels = profession
print("ground truth accuracy:", test_probe(oracle, get_acts, batches=batches, label_idx=0))
## test with labels = gender 
print("unintended feature accuracy:", test_probe(oracle, get_acts, batches=batches, label_idx=1))

probe


100%|██████████| 564/564 [00:54<00:00, 10.43it/s]


ambiguous test accuracy 0.921352207660675
ground truth accuracy: 0.9320276379585266
unintended feature accuracy: 0.4873271882534027


In [ ]:
# get worst-group accuracy of oracle probe
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(oracle, get_acts, batches=batches, label_idx=0))

In [8]:

probe, _ = train_probe(get_acts, label_idx=0)

probe


100%|██████████| 2798/2798 [04:46<00:00,  9.77it/s]


In [ ]:
# Save only the state dict (model parameters)
t.save(oracle.state_dict(), '../exp_redo/oracle_model_state.pth')

In [25]:
# Save only the state dict (model parameters)
t.save(probe.state_dict(), '../exp_redo/24_10_probe_model_state.pth')

#### load probe

In [4]:
# Define the model structure
probe = Probe(512)  # Replace with the actual model class

# Load the state dictionary
probe.load_state_dict(t.load('../exp_redo/24_10_probe_model_state.pth'))

# (Optional) Set the model to evaluation mode
probe.eval()

Probe(
  (net): Linear(in_features=512, out_features=1, bias=True)
)

In [9]:
## get testset for balanced data
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(probe, get_acts, batches=batches, label_idx=0))
print('Unintended feature accuracy:', test_probe(probe, get_acts, batches=batches, label_idx=1))

Ground truth accuracy: 0.6059907674789429
Unintended feature accuracy: 0.8790322542190552


In [8]:
## train with ambiguous data (gender biased data)
probe, _ = train_probe(get_acts, label_idx=0)
print('Ambiguous test accuracy:', test_probe(probe, get_acts, label_idx=0))
## get testset for balanced data
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(probe, get_acts, batches=batches, label_idx=0))
print('Unintended feature accuracy:', test_probe(probe, get_acts, batches=batches, label_idx=1))

probe


100%|██████████| 2798/2798 [05:19<00:00,  8.75it/s]


Ambiguous test accuracy: 0.9907063245773315
Ground truth accuracy: 0.6059907674789429
Unintended feature accuracy: 0.8790322542190552


In [ ]:
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(probe, get_acts, batches=batches, label_idx=0))

In [5]:
# loading dictionaries

# dictionary hyperparameters
dict_id = 10
expansion_factor = 64
dictionary_size = expansion_factor * activation_dim
DEVICE = 'cpu'
submodules = []
dictionaries = {}
path = "../dictionary_learning"
layer = 4 ## only retrieve c

submodules.append(model.gpt_neox.embed_in)
dictionaries[model.gpt_neox.embed_in] = AutoEncoder.from_pretrained(
    f'{path}/dictionaries/pythia-70m-deduped/embed/{dict_id}_{dictionary_size}/ae.pt',
    device=DEVICE
)
for i in range(layer + 1):
    submodules.append(model.gpt_neox.layers[i].attention)
    dictionaries[model.gpt_neox.layers[i].attention] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/attn_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

    submodules.append(model.gpt_neox.layers[i].mlp)
    dictionaries[model.gpt_neox.layers[i].mlp] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/mlp_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

    submodules.append(model.gpt_neox.layers[i])
    dictionaries[model.gpt_neox.layers[i]] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/resid_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

def metric_fn(model, labels=None):
    print(' :)))))))))) ')
    attn_mask = model.inputs[1]['attention_mask']
    acts = model.gpt_neox.layers[layer].output[0]
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    
    return t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )

# somehow middle layer is chosen for training the class
# middle layer - expected to be where higher level features are learnt?

In [10]:
submodules

[Embedding(50304, 512),
 GPTNeoXSdpaAttention(
   (rotary_emb): GPTNeoXRotaryEmbedding()
   (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
   (dense): Linear(in_features=512, out_features=512, bias=True)
   (attention_dropout): Dropout(p=0.0, inplace=False)
 ),
 GPTNeoXMLP(
   (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
   (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
   (act): GELUActivation()
 ),
 GPTNeoXLayer(
   (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_dropout): Dropout(p=0.0, inplace=False)
   (post_mlp_dropout): Dropout(p=0.0, inplace=False)
   (attention): GPTNeoXSdpaAttention(
     (rotary_emb): GPTNeoXRotaryEmbedding()
     (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
     (dense): Linear(in_features=512, out_features=512, bias=True)
     

In [9]:
len(dictionaries)

16

In [10]:
dictionaries.keys()

dict_keys([Embedding(50304, 512), GPTNeoXSdpaAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
  (dense): Linear(in_features=512, out_features=512, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
), GPTNeoXMLP(
  (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
  (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
  (act): GELUActivation()
), GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXSdpaAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_

In [9]:
def metric_fn(model, labels=None):
    attn_mask = model.inputs[1]['attention_mask']
    print('attn mask shape', attn_mask.shape)
    acts = model.gpt_neox.layers[layer].output[0]
    print('acts shape', acts.shape)
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    print('labels, metric', t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    ))
    return t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )

In [10]:
# TODO: 
# find most influential features
# n_batches = 25
from attribution import patching_effect
n_batches = 2
batch_size = 4

running_total = 0
nodes = None

## training data, for ambiguous (batch idx; text

for batch_idx, (clean, labels, _) in tqdm(enumerate(get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break
    print('-----------', batch_idx, clean, labels)

### estimate causal impact based on integrated gradients 
### take text, model, dictionaries, metric function

### blocker: metric fc, issue: datatype

    effects, _, _, _ = patching_effect(
        clean, # text
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    print('extracted effects')
    print(effects)
    with t.no_grad():
        if nodes is None:
            nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

## get most relevant nodes
nodes = {k : v / running_total for k, v in nodes.items()}

  0%|          | 0/2 [00:00<?, ?it/s]

----------- 0 ['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*', 'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center

  0%|          | 0/2 [00:00<?, ?it/s]

attn mask shape LanguageModelProxy (fetch_attr_72): 
acts shape LanguageModelProxy (fetch_attr_73): 
labels, metric LanguageModelProxy (where_18): 
attention mask torch.Size([40, 142])


RuntimeError: The size of tensor a (142) must match the size of tensor b (40) at non-singleton dimension 0

In [6]:
# find most influential features
# n_batches = 25
from attribution import patching_effect
n_batches = 2
batch_size = 4

running_total = 0
nodes = None

## training data, for ambiguous (batch idx; text

for batch_idx, (clean, labels, _) in tqdm(enumerate(get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break
    print('-----------', batch_idx, clean, labels)

### estimate causal impact based on integrated gradients 
### take text, model, dictionaries, metric function

### blocker: metric fc, issue: datatype

    effects, _, _, _ = patching_effect(
        clean, # text
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    print('extracted effects')
    print(effects)
    with t.no_grad():
        if nodes is None:
            nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

## get most relevant nodes
nodes = {k : v / running_total for k, v in nodes.items()}

  0%|          | 0/2 [00:00<?, ?it/s]You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


----------- 0 ['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*', 'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center

  0%|          | 0/2 [00:01<?, ?it/s]

 :)))))))))) 
attention mask torch.Size([40, 142])


RuntimeError: The size of tensor a (142) must match the size of tensor b (40) at non-singleton dimension 0

#### debug issues 4
- tuple: 
- .output (tuple vs parameter): what is 

- when does 4, 124 change to 40 124 : it is wrt to the batchsize * 10 steps; but how and why; and when does it happen? 

- how is submodule used in forward pass? (within tracing context..?)

- might be backward pass problem? (no.. causal mask is used in forward pass they say); then where is that forward pass 

- seem to be stuck, unable to loop the second submodule?

#### archive: somehow worked

In [ ]:
## tuple no attribute shape

In [29]:
# find most influential features
# n_batches = 25
n_batches = 2
batch_size = 4

running_total = 0
nodes = None

## training data, for ambiguous (batch idx; text

for batch_idx, (clean, labels, _) in tqdm(enumerate(get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break
    print('-----------', batch_idx, clean, labels)

### estimate causal impact based on integrated gradients 
### take text, model, dictionaries, metric function

### blocker: metric fc, issue: datatype

    effects, _, _, _ = patching_effect(
        clean, # text
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    print('extracted effects')
    print(effects)
    with t.no_grad():
        if nodes is None:
            nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

## get most relevant nodes
nodes = {k : v / running_total for k, v in nodes.items()}

  0%|          | 0/2 [00:00<?, ?it/s]

----------- 0 ['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*', 'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center

 50%|█████     | 1/2 [00:26<00:26, 26.37s/it]

----------- 1 ['He is the author of four books including the three-volume series The Place Setting, which explores Appalachian foodways. He directed and produced the documentary A Red Hot Dog Digest. Sauceman’s Food with Fred appears monthly on WJHL-TV, the CBS affiliate in Johnson City, Tennessee.', 'He received BSc and MSc degrees in computer science from the University of Hagen, Germany in 1994 and 1996, and a Dr.-Ing. and a Habilitation degree in computer engineering from the University of the Saarland at Saarbruecken, Germany in 2000 and 2002, respectively. Prior to joining the University of Hawaii, he worked for Advanced Micro Devices as a PMTS design engineer from 2007 to 2012, and he was a faculty member in the Computer Science and Engineering Department at SMU from 2000 to 2007. His research interests include computer arithmetic, computer architecture, formal methods, and hardware security.', 'She graduated with honors in 2010. Having more than 6 years of diverse experiences, 

100%|██████████| 2/2 [00:51<00:00, 25.51s/it]


In [32]:
# find most influential features
# n_batches = 25
n_batches = 2
batch_size = 2

running_total = 0
nodes = None

## training data, for ambiguous (batch idx; text

for batch_idx, (clean, labels, _) in tqdm(enumerate(get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break
    print('-----------', batch_idx, clean, labels)

### estimate causal impact based on integrated gradients 
### take text, model, dictionaries, metric function

### blocker: metric fc, issue: datatype

    effects, _, _, _ = patching_effect(
        clean, # text
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    print('extracted effects')
    print(effects)
    with t.no_grad():
        if nodes is None:
            nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

## get most relevant nodes
nodes = {k : v / running_total for k, v in nodes.items()}

## started w 2... then went to 20... 

  0%|          | 0/2 [00:00<?, ?it/s]

----------- 0 ['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*', 'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center

 50%|█████     | 1/2 [00:15<00:15, 15.09s/it]

----------- 1 ['Dr. Bigelow graduated from the University of Illinois-Urbana in May 2004 with a Ph.D. in Electrical Engineering. After completing his education, he was a Visiting Assistant Professor in the Electrical and Computer Engineering Department at the University of Illinois at Urbana-Champaign for a year. Dr. Bigelow was then an Assistant Professor in Electrical Engineering at the University of North Dakota for three years prior to coming to Iowa State University in August 2008. More About', "After completing his undergraduate degree at Clark University, he worked for two years as a speechwriter and deputy communications director for the Democratic Party in Massachusetts - while going to law school at night (fun stuff!). In 1998, Thor moved to Washington, DC to start graduate work at the Trachtenberg School of Public Policy and Public Administration at The George Washington University. At the same time, he began an eight-year career as a policy researcher at a series of think t

100%|██████████| 2/2 [00:31<00:00, 15.59s/it]


In [ ]:
### somehow worked but i dk how...

# what do these mean?
# attn mask shape LanguageModelProxy (fetch_attr_0): 
# acts shape LanguageModelProxy (fetch_attr_1): 
# labels, metric LanguageModelProxy (where_0):

In [110]:
len(nodes) # 

16

In [138]:
len(nodes.items())

16

In [26]:
nodes_list = []
for key, value in nodes.items(): 
    value = nodes[key].to_tensor()
    # print(value)
    nodes_list.append((str(key), value))
len(nodes_list)

16

In [28]:
import json 
with open('../exp_redo/nodes.json', 'w') as file:
    nodes_list = []
    for key, value in nodes.items(): 
        value = nodes[key].to_tensor().tolist()
        # print(value)
        nodes_list.append((str(key), value))
    json.dump(nodes_list, file, indent=4)

sidenote: what is envoy

In [ ]:
### debug seem to be the backward pass? (seem to not be also?)


### pe_ig

In [14]:
from activation_utils import SparseAct
# named tuple: Creates a new tuple subclass with named fields for each item
from collections import namedtuple
EffectOut = namedtuple('EffectOut', ['effects', 'deltas', 'grads', 'total_effect'])
from IPython import embed
DEBUGGING = True

if DEBUGGING:
    tracer_kwargs = {'validate' : True, 'scan' : True}
else:
    tracer_kwargs = {'validate' : False, 'scan' : False}

def _pe_ig(
        clean,
        patch,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(),
):
    
    # first run through a test input to figure out which hidden states are tuples
    is_tuple = {}
    with model.trace("_") as tracer:

        for submodule in submodules:
            # tracer.log(submodule)
            is_tuple[submodule] = type(submodule.output) == tuple

    hidden_states_clean = {}
    # to gather hidden states of a batch of inputs (8, embd)
    
    with model.trace(clean, **tracer_kwargs) as tracer, t.no_grad():
        # step through components of model, one by one
        # get dict/sae for the component
        # save features (32768); feature node activation
        # save residual (error term); error node activation
        for submodule in submodules:
            dictionary = dictionaries[submodule]
            # x = submodule.output ## NOTE: original
            tracer.log(submodule.output)
            x = submodule.output[0] ## NOTE: debugged by adding [0]
            # tracer.log(submodule)
            # tracer.log('x', x.shape)
            if is_tuple[submodule]:
                x = x[0]
            f = dictionary.encode(x) #32768
            # tracer.log('f', f.shape)
            x_hat = dictionary.decode(f) #512
            residual = x - x_hat
            # tracer.log('res', residual.shape)
            hidden_states_clean[submodule] = SparseAct(act=f.save(), res=residual.save())
        tracer.log('submodules done')
        ## just getting classification logits 
        metric_clean = metric_fn(model, **metric_kwargs).save()
        tracer.log('metric', metric_clean.shape)
    # {submodule : hidden state (saved as SparseAct class)}
    hidden_states_clean = {k : v.value for k, v in hidden_states_clean.items()}

    if patch is None:
        ## zeros
        hidden_states_patch = {
            k : SparseAct(act=t.zeros_like(v.act), res=t.zeros_like(v.res)) for k, v in hidden_states_clean.items()
        }
        total_effect = None
    else:
        ## hidden state of 'corrupted'; patched features
        hidden_states_patch = {}
        with model.trace(patch, **tracer_kwargs), t.no_grad():
            for submodule in submodules:
                dictionary = dictionaries[submodule]
                x = submodule.output
                if is_tuple[submodule]:
                    x = x[0]
                f = dictionary.encode(x)
                x_hat = dictionary.decode(f)
                residual = x - x_hat
                hidden_states_patch[submodule] = SparseAct(act=f.save(), res=residual.save())
            ## 
            metric_patch = metric_fn(model, **metric_kwargs).save()
        total_effect = (metric_patch.value - metric_clean.value).detach()
        hidden_states_patch = {k : v.value for k, v in hidden_states_patch.items()}

    effects = {}
    deltas = {}
    grads = {}
    for submodule in submodules:
        dictionary = dictionaries[submodule]
        ## TODO: seem to be stuck here
        clean_state = hidden_states_clean[submodule]
        print('clean', clean_state)
        patch_state = hidden_states_patch[submodule]
        print('patch', patch_state)
        with model.trace(**tracer_kwargs) as tracer:
            metrics = []
            fs = []
            print('in')
            ## linear approx
            for step in range(steps):
                alpha = step / steps
                f = (1 - alpha) * clean_state + alpha * patch_state
                f.act.retain_grad()
                f.res.retain_grad()
                fs.append(f)
                print(len(clean))

                with tracer.invoke(clean, scan=tracer_kwargs['scan']):
                    tracer.log(model.inputs) # 142
                    if is_tuple[submodule]:
                        submodule.output[0][:] = dictionary.decode(f.act) + f.res
                    else:
                        submodule.output = dictionary.decode(f.act) + f.res
                    metrics.append(metric_fn(model, **metric_kwargs))
            metric = sum([m for m in metrics])
            
            metric.sum().backward(retain_graph=True) # TODO : why is this necessary? Probably shouldn't be, contact jaden

        mean_grad = sum([f.act.grad for f in fs]) / steps
        mean_residual_grad = sum([f.res.grad for f in fs]) / steps
        grad = SparseAct(act=mean_grad, res=mean_residual_grad)
        delta = (patch_state - clean_state).detach() if patch_state is not None else -clean_state.detach()
        effect = grad @ delta

        effects[submodule] = effect
        deltas[submodule] = delta
        grads[submodule] = grad

    return EffectOut(effects, deltas, grads, total_effect)

In [ ]:
for text in clean: 
    tokens = model.tokenizer(text)
    print(len(tokens[0])) ## length based on longest input

132
83
99
142


In [ ]:
# model.tokenizer(text)

{'input_ids': [4553, 21006, 521, 31454, 4248, 387, 15319, 2499, 13, 344, 4307, 323, 767, 1107, 347, 247, 6519, 16360, 285, 19041, 10924, 6423, 323, 253, 9922, 7021, 275, 14070, 428, 1223, 1469, 281, 1569, 2143, 387, 2360, 313, 2337, 5017, 30970, 496, 8065, 13, 23077, 4395, 281, 5041, 13, 9087, 281, 1265, 16125, 789, 387, 253, 1535, 13261, 25210, 4726, 273, 5259, 11981, 285, 5259, 13500, 387, 380, 6086, 5041, 2499, 15, 2058, 253, 1072, 673, 13, 344, 3407, 271, 4314, 14, 2913, 5249, 347, 247, 3646, 22780, 387, 247, 2962, 273, 1158, 20470, 428, 436, 2908, 2716, 247, 9976, 347, 247, 2199, 6423, 1561, 253, 416, 5543, 11294, 434, 6875, 285, 10784, 11981, 6516, 15, 496, 436, 5350, 13, 344, 369, 253, 8624, 25881, 327, 7418, 2175, 5196, 323, 253, 5219, 3995, 7454, 273, 6875, 285, 10784, 11981, 313, 13792, 49, 481], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
clean

['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*',
 'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center Inc. Call Sh

In [16]:
_pe_ig(
        clean,
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(labels=labels),
)

TypeError: __index__ returned non-int (type LanguageModelProxy)

In [18]:
model.output

LanguageModelProxy (InterventionProtocol_16): 

In [16]:
def metric_fn(model, labels=None):
    attn_mask = model.inputs[1]['attention_mask']
    print('attn mask shape', attn_mask.shape)
    acts = model.gpt_neox.layers[layer].output[0]
    print('acts shape', acts.shape)
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    print('labels, metric', t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    ))
    return t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )


In [12]:
from activation_utils import SparseAct
# named tuple: Creates a new tuple subclass with named fields for each item
from collections import namedtuple
EffectOut = namedtuple('EffectOut', ['effects', 'deltas', 'grads', 'total_effect'])

DEBUGGING = True

if DEBUGGING:
    tracer_kwargs = {'validate' : True, 'scan' : True}
else:
    tracer_kwargs = {'validate' : False, 'scan' : False}

def _pe_ig(
        clean,
        patch,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(labels=labels),
):
    
    # first run through a test input to figure out which hidden states are tuples
    is_tuple = {}
    with model.trace("_") as tracer:

        for submodule in submodules:
            # tracer.log(submodule)
            is_tuple[submodule] = type(submodule.output) == tuple

    hidden_states_clean = {}

    # to gather hidden states of a batch of inputs (8, embd)
    
    with model.trace(clean, **tracer_kwargs) as tracer, t.no_grad():
        # step through components of model, one by one
        # get dict/sae for the component
        # save features (32768); feature node activation
        # save residual (error term); error node activation
        ## TODO: model submodules are traced here too?
        for submodule in submodules:
            dictionary = dictionaries[submodule]
            # x = submodule.output ## NOTE: original
            x = submodule.output[0] ## NOTE: debugged by adding [0]
            # tracer.log(submodule)
            # tracer.log('x', x.shape)
            if is_tuple[submodule]:
                x = x[0]
            f = dictionary.encode(x) #32768
            # tracer.log('f', f.shape)
            x_hat = dictionary.decode(f) #512
            residual = x - x_hat
            # tracer.log('res', residual.shape)
            hidden_states_clean[submodule] = SparseAct(act=f.save(), res=residual.save())
        tracer.log('submodules done')
        ## just getting classification logits 
        metric_clean = metric_fn(model, **metric_kwargs).save()
        tracer.log('metric', metric_clean.shape)
    # {submodule : hidden state (saved as SparseAct class)}
    hidden_states_clean = {k : v.value for k, v in hidden_states_clean.items()}

    if patch is None:
        ## zeros
        hidden_states_patch = {
            k : SparseAct(act=t.zeros_like(v.act), res=t.zeros_like(v.res)) for k, v in hidden_states_clean.items()
        }
        total_effect = None
    else:
        ## hidden state of 'corrupted'; patched features
        hidden_states_patch = {}
        with model.trace(patch, **tracer_kwargs), t.no_grad():
            for submodule in submodules:
                dictionary = dictionaries[submodule]
                x = submodule.output
                if is_tuple[submodule]:
                    x = x[0]
                f = dictionary.encode(x)
                x_hat = dictionary.decode(f)
                residual = x - x_hat
                hidden_states_patch[submodule] = SparseAct(act=f.save(), res=residual.save())
            ## 
            metric_patch = metric_fn(model, **metric_kwargs).save()
        total_effect = (metric_patch.value - metric_clean.value).detach()
        hidden_states_patch = {k : v.value for k, v in hidden_states_patch.items()}
    
    ## get linear approx
    effects = {}
    deltas = {}
    grads = {}
    count = 0
    for submodule in submodules:
        print('count -------------------------', count)
        # get sae 
        dictionary = dictionaries[submodule]
        ## TODO: seem to be stuck here
        clean_state = hidden_states_clean[submodule]
        print('clean', clean_state)
        patch_state = hidden_states_patch[submodule]
        print('patch', patch_state)
        with model.trace(**tracer_kwargs) as tracer:
            metrics = []
            fs = []
            print('in')
            ## linear approx
            ## for each module, get dictionary + activations (sparse rep for input)
            
            for step in range(steps):
                alpha = step / steps
                ## interpolation of act values (linear approx)
                ## for each step/interpolated value, get gradients 
                f = (1 - alpha) * clean_state + alpha * patch_state
                f.act.retain_grad()
                f.res.retain_grad()
                ## get interpolated feature values
                fs.append(f)
                print(f'{step}: clean - {clean}') # (4, 142)
                with tracer.invoke(clean, scan=tracer_kwargs['scan']):
                    # get output with interpolated feature values
                    if is_tuple[submodule]:
                        submodule.output[0][:] = dictionary.decode(f.act) + f.res
                    else:
                        submodule.output = dictionary.decode(f.act) + f.res
                    tracer.log(f'{step} : submodule output,{submodule.output.shape},{submodule.output}, ')
                    print('here')
                    metrics.append(metric_fn(model, **metric_kwargs))
            print('after 10 steps')

            metric = sum([m for m in metrics])
            tracer.log('metric', metric.shape)
            ## compute grad of metric wrt to features
            # metric.sum().backward(retain_graph=True)
            metric.sum().backward(retain_graph=True) # TODO : why is this necessary? Probably shouldn't be, contact jaden
        count += 1
        print('count', count)
        print('out of loop')
        mean_grad = sum([f.act.grad for f in fs]) / steps
        mean_residual_grad = sum([f.res.grad for f in fs]) / steps
        grad = SparseAct(act=mean_grad, res=mean_residual_grad)
        delta = (patch_state - clean_state).detach() if patch_state is not None else -clean_state.detach()
        effect = grad @ delta

        effects[submodule] = effect
        deltas[submodule] = delta
        grads[submodule] = grad

    return EffectOut(effects, deltas, grads, total_effect)

In [ ]:
labels

tensor([1, 1, 0, 0])

In [ ]:
model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map=DEVICE, dispatch=True)

In [13]:
_pe_ig(
        clean,
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(labels=labels),
)

TypeError: __index__ returned non-int (type LanguageModelProxy)

In [ ]:
## sae activations for each of the 16 submodules 
len(clean_hid_state)

16

In [ ]:
keys = []
for key in clean_hid_state.keys(): 
    print(key)
    break

In [ ]:
clean_hid_state[submodules[0]].act.shape
## [142, 32768]

torch.Size([142, 32768])

In [ ]:
clean_hid_state[submodules[0]].res.shape
## residual embed

torch.Size([142, 512])

In [ ]:
clean_hid_state[submodules[15]].act.shape
## [142, 32768]

torch.Size([4, 142, 32768])

In [ ]:
submodules[3]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXSdpaAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [ ]:
clean_hid_state

In [48]:
n_features = 0
for component_idx, effect in enumerate(nodes.values()):
    print(f"Component {component_idx}:")
    for idx in (effect > 0.1).nonzero():
        print(idx.item(), effect[idx].item())
        n_features += 1
print(f"total features: {n_features}")

AttributeError: 'NoneType' object has no attribute 'values'

#### to note: 
- how are the attributing it back to dataset? 

In [19]:
# interpret features
from dictionary_learning.interp import examine_dimension
# no circuitsviz; no z standard library
from dictionary_learning.utils import hf_dataset_to_generator

# change the following two lines to pick which feature to interpret
component_idx = 9
feat_idx = 31098

submodule = submodules[component_idx]
dictionary = dictionaries[submodule]

# interpret some features
data = hf_dataset_to_generator("monology/pile-uncopyrighted")
buffer = ActivationBuffer(
    data,
    model,
    submodule,
    d_submodule=512,
    refresh_batch_size=128, # decrease to fit on smaller GPUs
    n_ctxs=512, # decrease to fit on smaller GPUs
    device=DEVICE
)

out = examine_dimension(
    model,
    submodule,
    buffer,
    dictionary,
    dim_idx=feat_idx,
    n_inputs=256 # decrease to fit on smaller GPUs
)
print(out.top_tokens)
print(out.top_affected)
out.top_contexts

TypeError: __index__ returned non-int (type LanguageModelProxy)

In [46]:
pip install zstandard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached zstandard-0.23.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached zstandard-0.23.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install zstandard
## circuitsviz, umap, plotly


SyntaxError: invalid syntax (319378430.py, line 1)

In [ ]:
feats_to_ablate = {
    submodules[0] : [
        946, # 'his'
        # 5719, # 'research'
        7392, # 'He'
        # 10784, # 'Nursing'
        17846, # 'He'
        22068, # 'His'
        # 23079, # 'tastes'
        # 25904, # 'nursing'
        28533, # 'She'
        29476, # 'he'
        31461, # 'His'
        31467, # 'she'
        32081, # 'her'
        32469, # 'She'
    ],
    submodules[1] : [
        # 23752, # capitalized words, especially pronouns
    ],
    submodules[2] : [
        2995, # 'he'
        3842, # 'She'
        10258, # female names
        13387, # 'she'
        13968, # 'He'
        18382, # 'her'
        19369, # 'His'
        28127, # 'She'
        30518, # 'He'
    ],
    submodules[3] : [
        1022, # 'she'
        9651, # female names
        10060, # 'She'
        18967, # 'He'
        22084, # 'he'
        23898, # 'His'
        # 24799, # promotes surnames
        26504, # 'her'
        29626, # 'his'
        # 31201, # 'nursing'
    ],
    submodules[4] : [
        # 8147, # unclear, something with names
    ],
    submodules[5] : [
        24159, # 'She', 'she'
        25018, # female names
    ],
    submodules[6] : [
        4592, # 'her'
        8920, # 'he'
        9877, # female names
        12128, # 'his'
        15017, # 'she'
        # 17369, # contact info
        # 26969, # related to nursing
        30248, # female names
    ],
    submodules[7] : [
        13570, # promotes male-related words
        27472, # female names, promotes female-related words
    ],
    submodules[8] : [
    ],
    submodules[9] : [
        1995, # promotes female-associated words
        9128, # feminine pronouns
        11656, # promotes male-associated words
        12440, # promotes female-associated words
        # 14638, # related to contact information?
        29206, # gendered pronouns
        29295, # female names
        # 31098, # nursing-related words
    ],
    submodules[10] : [
        2959, # promotes female-associated words
        19128, # promotes male-associated words
        22029, # promotes female-associated words
    ],
    submodules[11] : [
    ],
    submodules[12] : [
        19558, # promotes female-associated words
        23545, # 'she'
        24806, # 'her'
        27334, # promotes male-associated words
        31453, # female names
    ],
    submodules[13] : [
        31101, # promotes female-associated words
    ],
    submodules[14] : [
    ],
    submodules[15] : [
        9766, # promotes female-associated words
        12420, # promotes female pronouns
        30220, # promotes male pronouns
    ]
}

print(f"Number of features to ablate: {sum(len(v) for v in feats_to_ablate.values())}")

In [ ]:
# putting feats_to_ablate in a more useful format
def n_hot(feats, dim=dictionary_size):
    out = t.zeros(dim, dtype=t.bool, device=DEVICE)
    for feat in feats:
        out[feat] = True
    return out

feats_to_ablate = {
    submodule : n_hot(feats) for submodule, feats in feats_to_ablate.items()
}


In [ ]:
# utilities for ablating features
is_tuple = {}
with t.no_grad(), model.trace("_"):
    for submodule in submodules:
        is_tuple[submodule] = type(submodule.output.shape) == tuple

def get_acts_ablated(
    text,
    model,
    submodules,
    dictionaries,
    to_ablate
):

    with t.no_grad(), model.trace(text, **tracer_kwargs):
        for submodule in submodules:
            dictionary = dictionaries[submodule]
            feat_idxs = to_ablate[submodule]
            x = submodule.output
            if is_tuple[submodule]:
                x = x[0]
            x_hat, f = dictionary(x, output_features=True)
            res = x - x_hat
            f[...,feat_idxs] = 0. # zero ablation
            if is_tuple[submodule]:
                submodule.output[0][:] = dictionary.decode(f) + res
            else:
                submodule.output = dictionary.decode(f) + res
        attn_mask = model.input[1]['attention_mask']
        act = model.gpt_neox.layers[layer].output[0]
        act = act * attn_mask[:, :, None]
        act = act.sum(1) / attn_mask.sum(1)[:, None]
        act = act.save()
    return act.value


# Accuracy after ablating features judged irrelevant by human annotators

In [ ]:
get_acts_abl = lambda text : get_acts_ablated(text, model, submodules, dictionaries, feats_to_ablate)

print('Ambiguous test accuracy:', test_probe(probe, get_acts_abl, label_idx=0))
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))
print('Spurious accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=1))

In [ ]:
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))

# Concept bottleneck probing baseline

In [ ]:
concepts = [    
    ' nurse',
    ' healthcare',
    ' hospital',
    ' patient',
    ' medical',
    ' clinic',
    ' triage',
    ' medication',
    ' emergency',
    ' surgery',
    ' professor',
    ' academia',
    ' research',
    ' university',
    ' tenure',
    ' faculty',
    ' dissertation',
    ' sabbatical',
    ' publication',
    ' grant',
]
# get concept vectors
with t.no_grad(), model.trace(concepts):
    concept_vectors = model.gpt_neox.layers[layer].output[0][:, -1, :].save()
concept_vectors = concept_vectors.value - concept_vectors.value.mean(0, keepdim=True)

def get_bottleneck(text):
    with t.no_grad(), model.trace(text, **tracer_kwargs):
        attn_mask = model.input[1]['attention_mask']
        acts = model.gpt_neox.layers[layer].output[0]
        acts = acts * attn_mask[:, :, None]
        acts = acts.sum(1) / attn_mask.sum(1)[:, None]
        # compute cosine similarity with concept vectors
        sims = (acts @ concept_vectors.T) / (acts.norm(dim=-1)[:, None] @ concept_vectors.norm(dim=-1)[None])
        sims = sims.save()
    return sims.value

In [ ]:
cbp_probe, _ = train_probe(get_bottleneck, label_idx=0, dim=len(concepts))
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(cbp_probe, get_bottleneck, batches=batches, label_idx=0))
print('Unintended feature accuracy:', test_probe(cbp_probe, get_bottleneck, batches=batches, label_idx=1))


In [ ]:
# get subgroup accuracies
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(cbp_probe, get_bottleneck, batches=batches, label_idx=0))

# Get skyline neuron performance

In [ ]:
# get neurons which are most influential for giving gender label
neuron_dicts = {
    submodule : IdentityDict(activation_dim).to(DEVICE) for submodule in submodules
}

n_batches = 25
batch_size = 4

running_total = 0
nodes = None

for batch_idx, (clean, _, labels) in tqdm(enumerate(get_data(train=True, ambiguous=False, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break

    effects, _, _, _ = patching_effect(
        clean,
        None,
        model,
        submodules,
        neuron_dicts,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    with t.no_grad():
        if nodes is None:
            nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

nodes = {k : v / running_total for k, v in nodes.items()}

In [ ]:
neurons_to_ablate = {}
total_neurons = 0
for component_idx, effect in enumerate(nodes.values()):
    print(f"Component {component_idx}:")
    neurons_to_ablate[submodules[component_idx]] = []
    for idx in (effect.act > 0.2135).nonzero():
        print(idx.item(), effect[idx].item())
        neurons_to_ablate[submodules[component_idx]].append(idx.item())
        total_neurons += 1
print(f"total neurons: {total_neurons}")

neurons_to_ablate = {
    submodule : n_hot([neuron_idx], dim=512) for submodule, neuron_idx in neurons_to_ablate.items()
}

In [ ]:
def get_acts_abl(text):
    with t.no_grad(), model.trace(text, **tracer_kwargs):
        for submodule in submodules:
            x = submodule.output
            if is_tuple[submodule]:
                x = x[0]
            x[...,neurons_to_ablate[submodule]] = x.mean(dim=(0,1))[...,neurons_to_ablate[submodule]] # mean ablation
            if is_tuple[submodule]:
                submodule.output[0][:] = x
            else:
                submodule.output = x

        attn_mask = model.input[1]['attention_mask']
        act = model.gpt_neox.layers[layer].output[0]
        act = act * attn_mask[:, :, None]
        act = act.sum(1) / attn_mask.sum(1)[:, None]
        act = act.save()
    return act.value

In [ ]:
print('Ambiguous test accuracy:', test_probe(probe, get_acts_abl, label_idx=0))
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))
print('Spurious accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=1))

In [ ]:
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))

# Get skyline feature performance

In [ ]:
# get features which are most useful for predicting gender label
n_batches = 25
batch_size = 4

running_total = 0
running_nodes = None

for batch_idx, (clean, _, labels) in tqdm(enumerate(get_data(train=True, ambiguous=False, batch_size=batch_size, seed=SEED)), total=n_batches):
    if batch_idx == n_batches:
        break

    effects, _, _, _ = patching_effect(
        clean,
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(labels=labels),
        method='ig'
    )
    with t.no_grad():
        if running_nodes is None:
            running_nodes = {k : len(clean) * v.sum(dim=1).mean(dim=0) for k, v in effects.items()}
        else:
            for k, v in effects.items():
                running_nodes[k] += len(clean) * v.sum(dim=1).mean(dim=0)
        running_total += len(clean)
    del effects, _
    gc.collect()

nodes = {k : v / running_total for k, v in running_nodes.items()}

In [ ]:
top_feats_to_ablate = {}
total_features = 0
for component_idx, effect in enumerate(nodes.values()):
    print(f"Component {component_idx}:")
    top_feats_to_ablate[submodules[component_idx]] = []
    for idx in (effect > 0.1107).nonzero():
        print(idx.item(), effect[idx].item())
        top_feats_to_ablate[submodules[component_idx]].append(idx.item())
        total_features += 1
print(f"total features: {total_features}")

In [ ]:
top_feats_to_ablate = {
    submodule : n_hot(feats) for submodule, feats in top_feats_to_ablate.items()
}
get_acts_abl = lambda text : get_acts_ablated(text, model, submodules, dictionaries, top_feats_to_ablate)

In [ ]:
print('Ambiguous test accuracy:', test_probe(probe, get_acts_abl, label_idx=0))
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))
print('Spurious accuracy:', test_probe(probe, get_acts_abl, batches=batches, label_idx=1))

In [ ]:
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(probe, get_acts_abl, batches=batches, label_idx=0))

# Retraining probe on activations after ablating features

In [ ]:
get_acts_abl = lambda text : get_acts_ablated(text, model, submodules, dictionaries, feats_to_ablate)

new_probe, _ = train_probe(get_acts_abl, label_idx=0)
print('Ambiguous test accuracy:', test_probe(new_probe, get_acts_abl, label_idx=0))
batches = get_data(train=False, ambiguous=False)
print('Ground truth accuracy:', test_probe(new_probe, get_acts_abl, batches=batches, label_idx=0))
print('Unintended feature accuracy:', test_probe(new_probe, get_acts_abl, batches=batches, label_idx=1))

In [ ]:
subgroups = get_subgroups(train=False, ambiguous=False)
for label_profile, batches in subgroups.items():
    print(f'Accuracy for {label_profile}:', test_probe(new_probe, get_acts_abl, batches=batches, label_idx=0))